# Combine Twitter Posts

## Step 1 - Format the Play by Play Tweet Data

In [137]:
import pandas as pd

df = pd.read_csv('all_pbp_tweets.tar.gz')

# Drop the rows that are just repeats of the header
df = df.drop(df.loc[df['created_at'] == 'created_at'].index)

# Drop columns that don't contain meaninful data
df = df.drop(['location','time_zone'], axis=1)

# Convert created_at to datetime
df['UTC_Datetime'] = pd.to_datetime(df['created_at'])

# Rename column to tweet id and make it the index
df = df.rename(columns={'cardinals_pbp_tweets.csv':'tweet_id'}).set_index('tweet_id')

# make the tweet_id numeric
df.index = pd.to_numeric(df.index)

# Drop a single annoying NA row
df = df.loc[df.index.dropna()] # Drop this annoying

# Make year and time columns
df['time'] = df['UTC_Datetime'].apply(lambda x: x.time())
df['date'] = df['UTC_Datetime'].apply(lambda x: x.date())

# Sort by the timestamp
df = df.sort_values('UTC_Datetime')

# Reorder the columns in the dataframe
df = df[['UTC_Datetime','date','time','screen_name','source','text']]

# Only Games from 2017 Season and name pbptweets
pbptweets = df.loc[df['UTC_Datetime'] > '04-Sep-2017']

In [138]:
pbptweets.head()

,UTC_Datetime,date,time,screen_name,source,text
tweet_id,,,,,,
9.059546e+17,2017-09-08 00:43:00,2017-09-08,00:43:00,patriots_pbp,Patriots Play By Play,C.Santos kicks 64 yards from KC 35 to NE 1. D....
9.059546e+17,2017-09-08 00:43:00,2017-09-08,00:43:00,chiefs_pbp,Chiefs Play By Play,C.Santos kicks 64 yards from KC 35 to NE 1. D....
9.059548e+17,2017-09-08 00:44:00,2017-09-08,00:44:00,patriots_pbp,Patriots Play By Play,1/2 (14:55) NE 12-Brady 18th season as Patriot...
9.059548e+17,2017-09-08 00:44:00,2017-09-08,00:44:00,patriots_pbp,Patriots Play By Play,2/2 T.Brady pass incomplete deep left to D.All...
9.059548e+17,2017-09-08 00:44:01,2017-09-08,00:44:01,chiefs_pbp,Chiefs Play By Play,1/2 (14:55) NE 12-Brady 18th season as Patriot...


## Step 2 Load the Win Percentage Data and Try to Match Up

** One thing to keep in mind - The Twitter timestamps are UTC. The game Dates are the date the game STARTED. If a game went past midnight - or changed dates in UTC times we might have a problem **

In [139]:
winpct = pd.read_csv('All_Games_Win_Pct.csv')

# Call the play text the same thing as the other dataframe
winpct['text'] = winpct['playtext']

# Convert date to datetime
winpct['date'] = pd.to_datetime(winpct['Game Date'])

In [145]:
merged = winpct.merge(pbptweets, how='left', on=['text'])

In [159]:
winpct['text'][7]

'C.Boswell kicks 65 yards from PIT 35 to end zone, Touchback.'

In [202]:
print(len(pbptweets['text']))
print(len(pbptweets['text'].drop_duplicates()))

79409
42722


In [210]:
# Keep only unique play text, keep the first timestamp
pbptweets = pbptweets.drop_duplicates(subset='text', keep='first')

In [216]:
import re
# Find all the tweets mentioning "SANTOS"
pbptweets.loc[pbptweets['text'].apply(lambda x: any(re.findall('Santos',x)))][['date','screen_name','text']]

,date,screen_name,text
tweet_id,,,
9.059546e+17,2017-09-08,patriots_pbp,C.Santos kicks 64 yards from KC 35 to NE 1. D....
9.059629e+17,2017-09-08,patriots_pbp,C.Santos extra point is GOOD Center-J.Winchest...
9.059637e+17,2017-09-08,patriots_pbp,C.Santos kicks 65 yards from KC 35 to end zone...
9.059780e+17,2017-09-08,chiefs_pbp,C.Santos kicks 57 yards from KC 35 to NE 8. D....
9.094799e+17,2017-09-17,chiefs_pbp,(11:26) C.Santos 34 yard field goal is GOOD Ce...
9.094799e+17,2017-09-17,eagles_pbp,C.Santos kicks 66 yards from KC 35 to PHI -1. ...
9.094851e+17,2017-09-17,eagles_pbp,(:21) C.Santos 39 yard field goal is GOOD Cent...
9.094852e+17,2017-09-17,eagles_pbp,C.Santos kicks 65 yards from KC 35 to end zone...
9.094935e+17,2017-09-17,eagles_pbp,C.Santos extra point is GOOD Center-J.Winchest...


In [212]:
print(len(pbptweets))

42722


In [217]:
# Find all the plays mentioning Santos
winpct.loc[winpct['text'].apply(lambda x: any(re.findall('Santos',x)))][['playId','homeWinPercentage','playtext','date']]

,playId,homeWinPercentage,playtext,date
1052,40095156644,0.754,C.Santos kicks 64 yards from KC 35 to NE 1. D....,2017-09-07
1085,400951566938,0.707,Demetrius Harris Pass From Alex Smith for 7 Yr...,2017-09-07
1086,400951566974,0.711,C.Santos kicks 65 yards from KC 35 to end zone...,2017-09-07
1142,4009515662317,0.724,Kareem Hunt Pass From Alex Smith for 3 Yrds C....,2017-09-07
1143,4009515662353,0.720,C.Santos kicks 57 yards from KC 35 to NE 8. D....,2017-09-07
1159,4009515662756,0.461,Tyreek Hill Pass From Alex Smith for 75 Yrds C...,2017-09-07
1160,4009515662792,0.468,C.Santos kicks 65 yards from KC 35 to end zone...,2017-09-07
1196,4009515663725,0.523,Kareem Hunt Pass From Alex Smith for 78 Yrds C...,2017-09-07
1197,4009515663761,0.531,C.Santos kicks 65 yards from KC 35 to end zone...,2017-09-07
1221,4009515664314,0.101,Kareem Hunt 4 Yard Rush C.Santos extra point i...,2017-09-07


In [213]:
# There are a lot more plays mentioning Stants than there are tweets? What's the deal?

In [219]:
santos_tweets = pbptweets.loc[pbptweets['text'].apply(lambda x: any(re.findall('Santos',x)))][['date','screen_name','text']]
santos_winpct = winpct.loc[winpct['text'].apply(lambda x: any(re.findall('Santos',x)))][['playId','homeWinPercentage','playtext','date']]

In [227]:
santos_tweets['date'] = pd.to_datetime(santos_tweets['date'])

In [248]:
santos_tweets.loc[santos_tweets['date'] == '2017-09-08']['text'].unique() # only 2017-09-08 Tweets

array(['C.Santos kicks 64 yards from KC 35 to NE 1. D.Lewis to NE 27 for 26 yards (K.Pierre-Louis D.Harris). #NEvsKC',
       'C.Santos extra point is GOOD Center-J.Winchester Holder-D.Colquitt. #NEvsKC',
       'C.Santos kicks 65 yards from KC 35 to end zone Touchback. #NEvsKC',
       'C.Santos kicks 57 yards from KC 35 to NE 8. D.Lewis to NE 18 for 10 yards (D.Sorensen). #NEvsKC'],
      dtype=object)

In [246]:
santos_winpct.loc[santos_winpct['date'] == '2017-09-07']['playtext'].unique()

array(['C.Santos kicks 64 yards from KC 35 to NE 1. D.Lewis to NE 27 for 26 yards (K.Pierre-Louis, D.Harris).',
       'Demetrius Harris Pass From Alex Smith for 7 Yrds C.Santos extra point is GOOD',
       'C.Santos kicks 65 yards from KC 35 to end zone, Touchback.',
       'Kareem Hunt Pass From Alex Smith for 3 Yrds C.Santos extra point is GOOD',
       'C.Santos kicks 57 yards from KC 35 to NE 8. D.Lewis to NE 18 for 10 yards (D.Sorensen).',
       'Tyreek Hill Pass From Alex Smith for 75 Yrds C.Santos extra point is GOOD',
       'Kareem Hunt Pass From Alex Smith for 78 Yrds C.Santos extra point is GOOD',
       'Kareem Hunt 4 Yard Rush C.Santos extra point is GOOD',
       'Charcandrick West 21 Yard Rush C.Santos extra point is GOOD',
       'C.Santos kicks 64 yards from KC 35 to NE 1. D.Lewis to NE 14 for 13 yards (U.Eligwe).'],
      dtype=object)

# Pull from one game to compare

In [295]:
pats_chiefs_nov8_tweets = pbptweets.loc[(pbptweets['screen_name'] == 'patriots_pbp') | 
                                        (pbptweets['screen_name'] == 'chiefs_pbp')]

In [254]:
pats_chiefs_nov8_tweets = pats_chiefs_nov8_tweets.loc[pats_chiefs_nov8_tweets['UTC_Datetime'] <= '09-10-2017']

In [262]:
pats_chiefs_nov8_plays = winpct.loc[winpct['Game Title'] == 'Chiefs vs. Patriots ']

In [266]:
# These are pretty close....
print(len(pats_chiefs_nov8_tweets), "Unique Tweets on Gameday")
print(len(pats_chiefs_nov8_plays), "Unique Plays on Gameday")

198 Unique Tweets on Gameday
193 Unique Plays on Gameday


In [274]:
pats_chiefs_nov8_tweets = pats_chiefs_nov8_tweets.reset_index().drop('tweet_id', axis=1)

In [282]:
pats_chiefs_nov8_plays = pats_chiefs_nov8_plays.reset_index()[['homeWinPercentage',
                                      'secondsLeft',
                                      'homeScore',
                                      'awayScore',
                                      'clockdisplayValue',
                                      'text']]

In [289]:
pats_chiefs_nov8_plays.head()

,homeWinPercentage,secondsLeft,homeScore,awayScore,clockdisplayValue,text
0,0.754,0,0,0,14:55,C.Santos kicks 64 yards from KC 35 to NE 1. D....
1,0.730,0,0,0,14:55,(14:55) T.Brady pass incomplete deep left to D...
2,0.748,0,0,0,14:49,(14:49) T.Brady pass short right to R.Burkhead...
3,0.769,0,0,0,14:14,(14:14) (Shotgun) J.White left guard to NE 43 ...
4,0.754,0,0,0,13:52,"(13:52) (No Huddle, Shotgun) J.White up the mi..."


In [290]:
joined = pats_chiefs_nov8_plays.join(pats_chiefs_nov8_tweets, how='outer', rsuffix='tweets')

In [294]:
joined[['text',
        'texttweets',
        'UTC_Datetime',
        'homeWinPercentage',
        'homeScore',
        'awayScore',
        'clockdisplayValue']].to_csv('Pats_Chiefs_TwittervsPlays.csv')